# 1. Business Task
Cyclistic is a company based on Chicago looking for how to keep growing in the future. Their business is based on different bike rental plans:

-Single-ride pass.

-Full-day pass.

-Annual membership.

We are going to analyse how annual members (Annual membership) differ from casual riders (Single pass and Full-day pass) using Cyclistic, in order to help the organization design marketing strategies aimed at converting casual riders into annual members. 

Annual members are more profitable than casual riders, so stakeholders believe that increasing the amount of annual riders is key for Cyclistic's growth in the future.

# 2. Data Preparation

The data used is internally collected monthly data from August 2021 to July 2022 by Cyclistic from their users. The data is stored by the company on AWS' S3 cloud object storaged solution in dot zip files.

https://divvy-tripdata.s3.amazonaws.com/index.html

In general the data contains anonymous user information regarding ride identification, starting and ending time of the ride, the starting and ending station of the ride, and the user type of the ride.

It's stored in the structured data format. However, there is some data missing in some columns.


#### Storing the data
1. The zip files were uncompressed, in an individual carpet for each time period, in the 'csv files' folder.

2. The csv files containg information about the Byke Trips were copied to the 'raw data' folder.



## Cleaning the data



#### Understanding the data
The "Trips" table contained the following columns:

- ride_id: String identification of the trip.

- rideable_type: Bike category used during the trip.

- started_at: Datetime of the starting time of the bike trip.

- ended_at: Datetime of the ending time of the bike trip.

- start_station_name: Name of the starting point station of the trip.

- start_station_id: Number identification of the starting point station of the trip.

- end_station_name: Name of the ending point station of the trip.

- end_station_id: Number identification of the ending point station of the trip.

- start_lat: Latitude point of the starting station of the trip.

- start_lng: Longitude point of the starting station of the trip.

- end_lat: Latitude point of the ending station of the trip.

- end_lng: Longitude point of the ending station of the trip.

- member_casual: Refers to the User Type of the trip. Category 'casual' refers to single-ride pass and full-day pass users. The 'member' category refers to users with annual memberships.

#### Data cleaning and formatting with Python and Pandas
The following changes were performed to the data

In [35]:
#Imported the necessary libraries
import pandas as pd
import numpy as np

In [36]:
#Load the data with pandas for each month (From August 2021 to July 2022)
trips_2021_08 = pd.read_csv('data/raw data/202108-divvy-tripdata.csv')
trips_2021_09 = pd.read_csv('data/raw data/202109-divvy-tripdata.csv')
trips_2021_10 = pd.read_csv('data/raw data/202110-divvy-tripdata.csv')
trips_2021_11 = pd.read_csv('data/raw data/202111-divvy-tripdata.csv')
trips_2021_12 = pd.read_csv('data/raw data/202112-divvy-tripdata.csv')
trips_2022_01 = pd.read_csv('data/raw data/202201-divvy-tripdata.csv')
trips_2022_02 = pd.read_csv('data/raw data/202202-divvy-tripdata.csv')
trips_2022_03 = pd.read_csv('data/raw data/202203-divvy-tripdata.csv')
trips_2022_04 = pd.read_csv('data/raw data/202204-divvy-tripdata.csv')
trips_2022_05 = pd.read_csv('data/raw data/202205-divvy-tripdata.csv')
trips_2022_06 = pd.read_csv('data/raw data/202206-divvy-tripdata.csv')
trips_2022_07 = pd.read_csv('data/raw data/202207-divvy-tripdata.csv')
trips_2022_08 = pd.read_csv('data/raw data/202201-divvy-tripdata.csv')

In [37]:
#Looking inside the data from August 2021 we can see there are rows with missing values in several columns
trips_2021_08.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804352 entries, 0 to 804351
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             804352 non-null  object 
 1   rideable_type       804352 non-null  object 
 2   started_at          804352 non-null  object 
 3   ended_at            804352 non-null  object 
 4   start_station_name  715894 non-null  object 
 5   start_station_id    715894 non-null  object 
 6   end_station_name    710237 non-null  object 
 7   end_station_id      710237 non-null  object 
 8   start_lat           804352 non-null  float64
 9   start_lng           804352 non-null  float64
 10  end_lat             803646 non-null  float64
 11  end_lng             803646 non-null  float64
 12  member_casual       804352 non-null  object 
dtypes: float64(4), object(9)
memory usage: 79.8+ MB


By using the info() function we know the data doesn't have any empty values for the datetime columns, but there's missing data regarding the station name's and geographical points.

In [38]:
#Adding the month is useful to latter concatenate them into a dataframe
trips_by_month = [
    trips_2021_08,
    trips_2021_09,
    trips_2021_10,
    trips_2021_11,
    trips_2021_12,
    trips_2022_01,
    trips_2022_02,
    trips_2022_03,
    trips_2022_04,
    trips_2022_05,
    trips_2022_06,
    trips_2022_07,
    trips_2022_08
    ]

In [39]:
#We can once more check that all the dataframes have the same number of columns
for month in trips_by_month:
    print(month.shape[1])

13
13
13
13
13
13
13
13
13
13
13
13
13


In [83]:
#Merged all the data into a single dataframe

trips_2021_2022 = pd.concat(trips_by_month)

In [41]:
trips_2021_2022.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6005233 entries, 0 to 103769
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   ride_id             6005233 non-null  object 
 1   rideable_type       6005233 non-null  object 
 2   started_at          6005233 non-null  object 
 3   ended_at            6005233 non-null  object 
 4   start_station_name  5128187 non-null  object 
 5   start_station_id    5128189 non-null  object 
 6   end_station_name    5067410 non-null  object 
 7   end_station_id      5067410 non-null  object 
 8   start_lat           6005233 non-null  float64
 9   start_lng           6005233 non-null  float64
 10  end_lat             5999557 non-null  float64
 11  end_lng             5999557 non-null  float64
 12  member_casual       6005233 non-null  object 
dtypes: float64(4), object(9)
memory usage: 641.4+ MB


In [42]:
trips_2021_2022.head(10)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,99103BB87CC6C1BB,electric_bike,2021-08-10 17:15:49,2021-08-10 17:22:44,NaN,NaN,NaN,NaN,41.77,-87.68,41.770000,-87.680000,member
1,EAFCCCFB0A3FC5A1,electric_bike,2021-08-10 17:23:14,2021-08-10 17:39:24,NaN,NaN,NaN,NaN,41.77,-87.68,41.770000,-87.630000,member
2,9EF4F46C57AD234D,electric_bike,2021-08-21 02:34:23,2021-08-21 02:50:36,NaN,NaN,NaN,NaN,41.95,-87.65,41.970000,-87.660000,member
3,5834D3208BFAF1DA,electric_bike,2021-08-21 06:52:55,2021-08-21 07:08:13,NaN,NaN,NaN,NaN,41.97,-87.67,41.950000,-87.650000,member
4,CD825CB87ED1D096,electric_bike,2021-08-19 11:55:29,2021-08-19 12:04:11,NaN,NaN,NaN,NaN,41.79,-87.60,41.770000,-87.620000,member
5,612F12C94A964F3E,electric_bike,2021-08-19 12:41:12,2021-08-19 12:47:47,NaN,NaN,NaN,NaN,41.81,-87.61,41.800000,-87.600000,member
6,C7435946FDFFA9B7,electric_bike,2021-08-19 12:21:50,2021-08-19 12:37:31,NaN,NaN,NaN,NaN,41.77,-87.62,41.810000,-87.610000,member
7,C67017767EED2251,electric_bike,2021-08-13 14:52:35,2021-08-13 14:58:16,NaN,NaN,Clark St & Grace St,TA1307000127,41.94,-87.64,41.950874,-87.659146,member
8,ABC4532F2B4983AB,electric_bike,2021-08-17 18:23:55,2021-08-17 18:24:13,NaN,NaN,NaN,NaN,41.92,-87.66,41.920000,-87.660000,member
9,82437E52DC3B9A8A,electric_bike,2021-08-04 12:50:53,2021-08-04 13:08:20,NaN,NaN,NaN,NaN,41.74,-87.53,41.740000,-87.530000,member


We will start by checking if there aren't any trips with duplicated ride_id's

In [91]:
original_number_ride_ids = len(trips_2021_2022['ride_id'])
non_duplicated_ride_ids = len(trips_2021_2022['ride_id'].drop_duplicates())
number_of_duplicated_ids = original_number_ride_ids - non_duplicated_ride_ids
number_of_duplicated_ids # 103770 trips with duplicated ride_id's

103770

Since ride_id's are supposed to be unique for each trip, we will drop the rows that have duplicated ride_id's

In [92]:
trips_2021_2022.drop_duplicates(subset=['ride_id'], inplace=True)

We know there's a lot of missing data regarding station names and geographical points.

Since we it could be useful to use geographical data to map each the Stations by trip

we will delete the rows that have missing data in the

'start_lat', 'start_lng', 'end_lat', and 'end_lng' columns.

In [43]:
trips_2021_2022 = trips_2021_2022[trips_2021_2022['start_lat'].notna()]
trips_2021_2022 = trips_2021_2022[trips_2021_2022['start_lng'].notna()]
trips_2021_2022 = trips_2021_2022[trips_2021_2022['end_lat'].notna()]
trips_2021_2022 = trips_2021_2022[trips_2021_2022['end_lng'].notna()]

#If we we didn't want to plot the stations by trips we would drop all the following columns:
#trips_2021_2022.drop(columns=['start_lat', 'start_lng', 'end_lat', 'end_lng'], inplace=True)

In [44]:
trips_2021_2022.info(verbose=True, show_counts=True)
#By dropping the empty geographical points we've reduced the data by 0.1%.
# We've gone from 6005233 rows to 5999557 rows.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5999557 entries, 0 to 103769
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   ride_id             5999557 non-null  object 
 1   rideable_type       5999557 non-null  object 
 2   started_at          5999557 non-null  object 
 3   ended_at            5999557 non-null  object 
 4   start_station_name  5122511 non-null  object 
 5   start_station_id    5122513 non-null  object 
 6   end_station_name    5067410 non-null  object 
 7   end_station_id      5067410 non-null  object 
 8   start_lat           5999557 non-null  float64
 9   start_lng           5999557 non-null  float64
 10  end_lat             5999557 non-null  float64
 11  end_lng             5999557 non-null  float64
 12  member_casual       5999557 non-null  object 
dtypes: float64(4), object(9)
memory usage: 640.8+ MB


In [45]:
#Since we're instered to know the behaviour of the users, 
# we will drop the rows with empty data in the 'started_at', 'ended_at', and 'ride_id' columns
trips_2021_2022 = trips_2021_2022[trips_2021_2022['started_at'].notna()]
trips_2021_2022 = trips_2021_2022[trips_2021_2022['ended_at'].notna()]
trips_2021_2022 = trips_2021_2022[trips_2021_2022['ride_id'].notna()]

In [46]:
trips_2021_2022.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5999557 entries, 0 to 103769
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   ride_id             5999557 non-null  object 
 1   rideable_type       5999557 non-null  object 
 2   started_at          5999557 non-null  object 
 3   ended_at            5999557 non-null  object 
 4   start_station_name  5122511 non-null  object 
 5   start_station_id    5122513 non-null  object 
 6   end_station_name    5067410 non-null  object 
 7   end_station_id      5067410 non-null  object 
 8   start_lat           5999557 non-null  float64
 9   start_lng           5999557 non-null  float64
 10  end_lat             5999557 non-null  float64
 11  end_lng             5999557 non-null  float64
 12  member_casual       5999557 non-null  object 
dtypes: float64(4), object(9)
memory usage: 640.8+ MB


In [47]:
#Converting the 'started_at' and 'ended_at' columns to datatime format
trips_2021_2022['started_at'] = pd.to_datetime(trips_2021_2022['started_at'])
trips_2021_2022['ended_at'] = pd.to_datetime(trips_2021_2022['ended_at'])

In [48]:
#Now we've changed the data type of the time columns
trips_2021_2022.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5999557 entries, 0 to 103769
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    object        
 6   end_station_name    object        
 7   end_station_id      object        
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
dtypes: datetime64[ns](2), float64(4), object(7)
memory usage: 640.8+ MB


Now we want to know how many different Starting and Ending stations are saved in the data 

In [49]:
#We can see there are 1380 different Starting Stations
trips_2021_2022['start_station_name'].value_counts().shape

(1381,)

In [53]:
#We can see there are 1392 different Ending Stations
trips_2021_2022['end_station_name'].value_counts().shape

(1396,)

In [50]:
#Not all Starting Stations are identifiable by their id
trips_2021_2022['start_station_id'].value_counts().shape

(1226,)

In [54]:
#Not all Ending Stations are identifiable by their id
trips_2021_2022['end_station_id'].value_counts().shape

(1236,)

Let's find if there's some error in the difference between the amount of Start Station names and End Station names.


In [73]:
#We sort the values by alphabetic order and look for unique values
start_names = trips_2021_2022.sort_values(by=['start_station_name'], ascending=True)
end_names = trips_2021_2022.sort_values(by=['end_station_name'], ascending=True)
start_names = set(start_names.start_station_name)
end_names = set(end_names.end_station_name)

In [76]:
#Let's find if there's some error in the difference between the amount of Start Station names and End Station names
start_names.difference(end_names)

#There's at least one who's Start Names seems weird, '351'

{'351',
 'Buffalo Ave & 133rd St',
 'Hale Ave & 111th St',
 'Keef Ave & South Chicago Ave',
 'Lamon Ave & Archer Ave',
 'Oglesby Ave & 105th St',
 'Public Rack - La Moyne Ave and Leamington Ave',
 'Public Rack - Western Ave & 98th St',
 'WestChi'}

In [79]:
trips_2021_2022[trips_2021_2022['start_station_name']=='351']

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
80741,5E181D51F7C391F4,electric_bike,2021-08-04 16:31:29,2021-08-04 16:48:33,351,351,Mulligan Ave & Wellington Ave,351,41.93,-87.78,41.93,-87.78,casual
606661,3036610505F382EF,electric_bike,2021-08-07 23:59:38,2021-08-08 00:37:58,351,351,NaN,NaN,41.93,-87.78,41.94,-87.79,casual


In [77]:
#There doesn't seem to be a problem with the names of the End Stations
end_names.difference(start_names)


{'10101 S Stony Island Ave',
 'Bennett Ave & 96th St',
 'Corliss Ave & 133rd St',
 'DIVVY CASSETTE REPAIR MOBILE STATION',
 'Ellis Ave & 132nd Pl',
 'Halsted St & 64th St',
 'James Madison School',
 'Keeler Ave & 26th St',
 'Kildare Ave & Division Ave',
 'Linder Ave & Archer Ave',
 'Michele Clark Magnet High School',
 'Normal Blvd & 61st Pl',
 'Orville T Bright School',
 'Park Manor Elementary School',
 'Parnell Ave & 98th St',
 'Percy Julian HS',
 'Public Rack - California Ave & Touhy Ave - SW',
 'Public Rack - Foster Ave & Drake Ave',
 'Public Rack - Keeler Ave & 26th St',
 'Pullman Community Center',
 'South Shore High School',
 'The Montessori School of Englewood',
 'Torrence Ave & 98th St',
 'William Rainey Harper High School'}

In [51]:
#We know check the amount of missing values in the starting station name column
trips_2021_2022['start_station_name'].isna().sum() 
#850166 rows with null values
#That's 14.17% of the data

877046

In [55]:
#We know check the amount of missing values in the ending station name column
trips_2021_2022['end_station_name'].isna().sum() 
#932147 rows with null values
#That's 15.54% of the data

932147

Since we can't ask the stakeholders if they're willing to delete at least 14.96% of the data for the analysis, it's up to us to decide.

One way we can get around this problem is trying to match the most amount of the station names given their longitude and latitude positions.

We can fill out the missing values doing the following. For each row that doesn't have a station name, and does have longitude and latitude values, we can check for rows that have matching longitud and latitude values and don't have their station name missing and copy their station name into the row. This can be done for the start_station_name and end_station_name.

In [34]:
trips_2021_2022.groupby(['end_lat', 'end_lng'])

In [62]:
df_with_end_names = trips_2021_2022[trips_2021_2022['end_station_name'].notna()]
df_with_end_names = df_with_end_names[['end_station_name', 'end_lat', 'end_lng']]

end_names= df_with_end_names['end_station_name'].map(df_with_end_names)
end_names

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
df_missing_end_names = trips_2021_2022[trips_2021_2022['end_station_name'].isna()]

for indx, row in df_missing_end_names.iterrows():
    print(indx)

In [ ]:
    cond1 = trips_2021_2022['start_lat'] == i_row['start_lat']
    cond2 = trips_2021_2022['start_lng'] == i_row['start_lng']
    cond3 = trips_2021_2022['start_station_name'].notna()
    rows_with_lat_long = trips_2021_2022.loc[cond1 & cond2 & cond3]
    if rows_with_lat_long:
        print(rows_with_lat_long.first())

In [95]:
#We add a new column called 'trip_duration', since this is an important feature we can look into by user type
trips_2021_2022['trip_duration'] = trips_2021_2022['ended_at'] - trips_2021_2022['started_at']

In [96]:
#Checking the new column
trips_2021_2022.head(10)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_duration
0,99103BB87CC6C1BB,electric_bike,2021-08-10 17:15:49,2021-08-10 17:22:44,NaN,NaN,NaN,NaN,41.77,-87.68,41.770000,-87.680000,member,0 days 00:06:55
1,EAFCCCFB0A3FC5A1,electric_bike,2021-08-10 17:23:14,2021-08-10 17:39:24,NaN,NaN,NaN,NaN,41.77,-87.68,41.770000,-87.630000,member,0 days 00:16:10
2,9EF4F46C57AD234D,electric_bike,2021-08-21 02:34:23,2021-08-21 02:50:36,NaN,NaN,NaN,NaN,41.95,-87.65,41.970000,-87.660000,member,0 days 00:16:13
3,5834D3208BFAF1DA,electric_bike,2021-08-21 06:52:55,2021-08-21 07:08:13,NaN,NaN,NaN,NaN,41.97,-87.67,41.950000,-87.650000,member,0 days 00:15:18
4,CD825CB87ED1D096,electric_bike,2021-08-19 11:55:29,2021-08-19 12:04:11,NaN,NaN,NaN,NaN,41.79,-87.60,41.770000,-87.620000,member,0 days 00:08:42
5,612F12C94A964F3E,electric_bike,2021-08-19 12:41:12,2021-08-19 12:47:47,NaN,NaN,NaN,NaN,41.81,-87.61,41.800000,-87.600000,member,0 days 00:06:35
6,C7435946FDFFA9B7,electric_bike,2021-08-19 12:21:50,2021-08-19 12:37:31,NaN,NaN,NaN,NaN,41.77,-87.62,41.810000,-87.610000,member,0 days 00:15:41
7,C67017767EED2251,electric_bike,2021-08-13 14:52:35,2021-08-13 14:58:16,NaN,NaN,Clark St & Grace St,TA1307000127,41.94,-87.64,41.950874,-87.659146,member,0 days 00:05:41
8,ABC4532F2B4983AB,electric_bike,2021-08-17 18:23:55,2021-08-17 18:24:13,NaN,NaN,NaN,NaN,41.92,-87.66,41.920000,-87.660000,member,0 days 00:00:18
9,82437E52DC3B9A8A,electric_bike,2021-08-04 12:50:53,2021-08-04 13:08:20,NaN,NaN,NaN,NaN,41.74,-87.53,41.740000,-87.530000,member,0 days 00:17:27


In [97]:
#We can see all columns
trips_2021_2022.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5999557 entries, 0 to 103769
Data columns (total 14 columns):
 #   Column              Non-Null Count    Dtype          
---  ------              --------------    -----          
 0   ride_id             5999557 non-null  object         
 1   rideable_type       5999557 non-null  object         
 2   started_at          5999557 non-null  datetime64[ns] 
 3   ended_at            5999557 non-null  datetime64[ns] 
 4   start_station_name  5122511 non-null  object         
 5   start_station_id    5122513 non-null  object         
 6   end_station_name    5067410 non-null  object         
 7   end_station_id      5067410 non-null  object         
 8   start_lat           5999557 non-null  float64        
 9   start_lng           5999557 non-null  float64        
 10  end_lat             5999557 non-null  float64        
 11  end_lng             5999557 non-null  float64        
 12  member_casual       5999557 non-null  object         
 13

In [98]:
#We filter the data by trips that lasted more than 1 minute
trips_2021_2022 = trips_2021_2022.loc[(trips_2021_2022['trip_duration'] > '00:01:00')]

In [99]:
trips_2021_2022.info(verbose=True, show_counts=True)
#We've reduced the number of rows from 6005233 to 5897752

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5892095 entries, 0 to 103769
Data columns (total 14 columns):
 #   Column              Non-Null Count    Dtype          
---  ------              --------------    -----          
 0   ride_id             5892095 non-null  object         
 1   rideable_type       5892095 non-null  object         
 2   started_at          5892095 non-null  datetime64[ns] 
 3   ended_at            5892095 non-null  datetime64[ns] 
 4   start_station_name  5041929 non-null  object         
 5   start_station_id    5041931 non-null  object         
 6   end_station_name    4994566 non-null  object         
 7   end_station_id      4994566 non-null  object         
 8   start_lat           5892095 non-null  float64        
 9   start_lng           5892095 non-null  float64        
 10  end_lat             5892095 non-null  float64        
 11  end_lng             5892095 non-null  float64        
 12  member_casual       5892095 non-null  object         
 13